In [ ]:
import os
import shutil
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split

In [ ]:
train_df = pd.read_csv('./dataset/train_df.csv')
train_df.head()

In [ ]:
train_df.tail()

In [ ]:
data_pre = train_df["file_name"].to_numpy()
target = train_df["label"].to_numpy()
train_x, valid_x, train_y, valid_y = train_test_split(
    data_pre, target, test_size=0.2, stratify=target
)
print(train_x.shape,valid_x.shape)

In [ ]:
train_path = "./dataset/main_directory/train/"
valid_path = "./dataset/main_directory/valid/"

if not os.path.exists(train_path):
        os.mkdir(train_path)
if not os.path.exists(valid_path):
        os.mkdir(valid_path)

for label in train_df["label"].unique():
    if not os.path.exists(train_path+label):
        os.mkdir(train_path+label)
    if not os.path.exists(valid_path+label):
        os.mkdir(valid_path+label)       

In [ ]:
for file, label in zip(train_x,train_y):
    src = "./dataset/train/"+file
    dst = train_path + label + '/' + label + '_train_' + file
    shutil.copy(src, dst)

In [ ]:
for file, label in zip(valid_x,valid_y):
    src = "./dataset/train/"+file
    dst = valid_path + label + '/' + label + '_train_' + file
    shutil.copy(src, dst)

In [ ]:
INPUT_SHAPE = (300,300,3)

In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=5,
    vertical_flip=True,
    fill_mode='nearest'
)
train_generator = train_datagen.flow_from_directory(
    directory= train_path,
    target_size= INPUT_SHAPE[:2],
    save_to_dir='./gen_train/',
    batch_size=4,
    class_mode = "categorical"
)

In [ ]:
valid_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=5,
    vertical_flip=True,
    fill_mode='nearest'
)
valid_generator = valid_datagen.flow_from_directory(
    directory= valid_path,
    target_size= INPUT_SHAPE[:2],
    save_to_dir='./gen_valid/',
    batch_size=4,
    class_mode = "categorical"
)

In [ ]:
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    directory='./dataset/test',
    save_to_dir='./gen_test/',
    target_size= INPUT_SHAPE[:2],
    batch_size=4,
    class_mode= None,
    shuffle=False
)

In [ ]:
transfer_model = keras.applications.Xception(weights='imagenet',
                                         include_top = False,
                                         input_shape = INPUT_SHAPE)
transfer_model.trainable = False
transfer_model.summary()

In [ ]:
finetune_model = keras.Sequential()
finetune_model.add(transfer_model)
finetune_model.add(keras.layers.Flatten())
finetune_model.add(keras.layers.Dense(200, activation='relu'))
finetune_model.add(keras.layers.Dense(88, activation='softmax'))

finetune_model.summary()

In [ ]:
finetune_model.compile(loss='categorical_crossentropy',
                      optimizer=keras.optimizers.Adam(learning_rate=0.0002),
                       metrics='accuracy')                    

In [ ]:
if not os.path.exists('./gen_train'):
    os.mkdir('./gen_train')
if not os.path.exists('./gen_valid'):
    os.mkdir('./gen_valid')
if not os.path.exists('./gen_test'):
    os.mkdir('./gen_test')

tf.random.set_seed(0)

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

hist = finetune_model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2)

In [ ]:
len(test_generator)

In [ ]:
test_generator.resete()
result = finetune_model.predict(test_generator)
result.shape

In [ ]:
label = os.listdir('./dataset/main_directory/train/')
len(label)

In [ ]:
target_labels = []
for i in range(result.shape[0]):
    j = result[i].argmax()
    print(f"{i}번째 예측:",j,label[j])
    target_labels.append(label[j])    

In [ ]:
df = pd.DataFrame(target_labels)
df.columns = ["label"]
df

In [ ]:
df.to_csv('submission.csv')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(hist.history['loss'], c='r')
plt.plot(hist.history['val_loss'], c='b')

plt.show()

In [ ]:
finetune_model.save("my_model.h5")